In [ ]:
import copy

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from sherpa.utils import poisson_noise, dataspace1d, dataspace2d
from sherpa.models import Gauss1D, Const1D, Gauss2D, Const2D
from sherpa.data import Data1DInt, Data2DInt
from sherpa.fit import Fit
from sherpa.stats import Cash
from sherpa.optmethods import LevMar, NelderMead, MonCar
from sherpa.plot import DataPlot, ModelPlot

In [ ]:
xlo, xhi, y = dataspace1d(-5, 5, 0.25)
crossdisp = Data1DInt(name="crossdisp", xlo=xlo, xhi=xhi, y=y)
gline = Gauss1D(name="gline")
bkg = Const1D(name="bkg")

bkg.c0 = 2
gline.fwhm = 3
gline.ampl = 15
gline.pos = 0

orig_model = gline + bkg

In [ ]:
# Make an new, independent model that we can use to fit without changing the numbers in the original model
# However, set starting values close, so that the fit has a good chance of converging.
fgline = Gauss1D(name="fgline")
fbkg = Const1D(name="fbkg")

fbkg.c0 = 2.1
fbkg.c0.min = 0
fgline.fwhm = 3.0
fgline.ampl = 21.6
fgline.ampl.min = 0
fgline.pos = 0
# Assume we know the FWHM from the CALDB and the pos because we centered the source.
fgline.fwhm.frozen = True
fgline.pos.frozen = True

fmodel = fgline + fbkg

fitter = Fit(crossdisp, fmodel, stat=Cash(), method=LevMar())
confidencer = Confidence()

In [ ]:
est_res.parvals[0], est_res.parmins[0], est_res.parmaxes[0]

In [ ]:
n = 10

full_model_results = np.zeros(n, 3)

for i in range(n):
    y = poisson_noise(orig_model(xlo, xhi))
    crossdisp.set_dep(y)
    # reset the model to the something sensible, in case the previous fit messed up
    fbkg.c0 = 2.1
    fbkg.c0.min = 0
    fgline.fwhm = 3.
    fgline.ampl = 21.6

    fitter.fit()
    fit_res = fitter.est_errors()
    full_model_results[i, :] = [fit_res.parvals[0], fit_res.parmins[0], fit_res.parmaxes[0]]
    

In [ ]:
y = poisson_noise(crossdisp.eval_model(orig_model))

In [ ]:
plt.plot(xlo, y)

In [ ]:
crossdisp.set_dep(y)

In [ ]:
crossdisp

In [ ]:
# Make an new, independent model that we can use to fit without changing the numbers in the original model
# However, set starting values close, so that the fit has a good chance of converging.
fgline = Gauss1D(name="fgline")
fbkg = Const1D(name="fbkg")

fbkg.c0 = 2.1
fbkg.c0.min = 0
fgline.fwhm = 3.
fgline.ampl = 21.6
fgline.ampl.min = 0
fgline.pos = 0
# Assume we know the FWHM from the CALDB and the pos because we centered the source.
fgline.fwhm.frozen = True
fgline.pos.frozen = True

fmodel = fgline + fbkg

In [ ]:
fmodel

In [ ]:
f = Fit(crossdisp, fmodel, stat=Cash(), method=LevMar())

In [ ]:
f.fit()

In [ ]:
dplot = DataPlot()
dplot.prepare(f.data)
mplot = ModelPlot()
mplot.prepare(f.data, f.model)
dplot.plot()
mplot.overplot()

In [ ]:
def bound(interval):
    # 2 sqrt(2 ln(2)) = 2.344 which converts from FWHM to sigma for a Gaussian
    return norm(scale=gline.fwhm.val / 2.355).interval(interval)[1]

In [ ]:
bxlo=[xlo.min(), -bound(0.8), bound(0.95)]
bxhi=[-bound(0.95), bound(0.8), xhi.max()]

by = [ y[(xlo >= xl) & (xhi <= xh)].sum() for xl, xh in zip(bxlo, bxhi)]

bcross = Data1DInt(name='bcross', xlo=bxlo, xhi=bxhi, y=by)

In [ ]:
bxlo, bxhi

In [ ]:
bf = Fit(bcross, fmodel, stat=Cash(), method=LevMar())

In [ ]:
bf.fit()

In [ ]:
dplot = DataPlot()
dplot.plot_prefs['xerrorbars'] = True
dplot.prepare(bf.data)
mplot = ModelPlot()
mplot.prepare(bf.data, bf.model)
dplot.plot()
mplot.overplot()

## Set up a toy 2D model

In [ ]:
from sherpa.astro.data import DataIMGInt

In [ ]:
xrange = np.arange(21.4, 21.7, 0.05)
yrange = np.arange(-.15, .15001, 0.01)

print(len(xrange), len(yrange))

x0lo, x1lo = np.meshgrid(xrange[:-1], yrange[:-1])
x0hi, x1hi = np.meshgrid(xrange[1:], yrange[1:])

x0lo = x0lo.flatten()
x1lo = x1lo.flatten()
x0hi = x0hi.flatten()
x1hi = x1hi.flatten()


gline = Gauss2D(name="gline")
bkg = Const2D(name="bkg")

bkg.c0 = 0
gline.fwhm = 2
gline.ampl = 1115
gline.ypos = 0
gline.ypos.frozen = True
gline.xpos = 21.6
gline.ellip = 0
gline.theta = np.pi / 2

orig_model = gline + bkg

In [ ]:

image = DataIMGInt("binned_image",
                x0lo=x0lo, x1lo=x1lo,
                    x0hi=x0hi, x1hi=x1hi,
                    y=np.zeros_like(x0lo), 
                    shape=[len(xrange)-1, len(yrange)-1])

In [ ]:
image.set_dep(poisson_noise(orig_model(x0lo=x0lo, x1lo=x1lo, x0hi=x0hi, x1hi=x1hi)))

In [ ]:
image

In [ ]:
        >>> import numpy as np
        >>> x = np.random.normal(size=10000, loc=21.6, scale=0.03)
        >>> y = np.random.normal(size=10000, scale=0.03)
        >>> xrange = np.arange(21.4, 21.7, 0.01)
        >>> yrange = np.arange(-.25, .250001, 0.01)
        >>> hist, x0edges, x1edges = np.histogram2d(x, y, bins=(xrange, yrange))
        >>> x0lo, x1lo = np.meshgrid(x0edges[:-1], x1edges[:-1])
        >>> x0hi, x1hi = np.meshgrid(x0edges[1:], x1edges[1:])
        >>> image = DataIMGInt("binned_image",
        ...                    x0lo=x0lo.flatten(), x1lo=x1lo.flatten(),
        ...                    x0hi=x0hi.flatten(), x1hi=x1hi.flatten(),
        ...                    y=hist.flatten(), shape=hist.shape)

In [ ]:
print(len(xrange), len(yrange))

In [ ]:
image

In [ ]:
f2d = Fit(data=image, model=orig_model, stat=Cash(), method=MonCar())

In [ ]:
# Tuen te parameters until I'm close, because I'm guessing te coordinate system here
bkg.c0 = 0
bkg.c0.freeze()
gline.fwhm = .05
gline.fwhm.freeze()
gline.ampl = 115
gline.ypos = 21.55
gline.ypos.frozen = False
gline.xpos = 0.1
gline.xpos.frozen = False
gline.ellip = 0
gline.theta = 0


In [ ]:
f2d.fit()

In [ ]:
f2d.model

In [ ]:
plt.imshow(gline(x0lo=image.x0lo, x1lo=image.x1lo, 
                 x0hi=image.x0hi, x1hi=image.x1hi).reshape(50, 30).T)

In [ ]:
plt.imshow(hist)

In [ ]:
out = image.get_img(orig_model)

In [ ]:
plt.imshow(out[0])

In [ ]:
plt.imshow(out[1])

## This is the part that I used for the proposal

So, it seems I have to make an example that is square, place the signal in the middle, and make the Gaussian round, not elliptical. Fortunately, with just one spectral line, I can always do this rescaling without loss of generality. When I make plots for the proposal, I can still rescale image axes using the xlim, aspect and extend to make it look closer to the original XMM situation.  

In [ ]:
from sherpa.models import NormGauss1D, NormGauss2D

from sherpa.data import DataSimulFit
from sherpa.models.model import SimulFitModel

In [ ]:
bkg = Const2D(name="bkg")
gline = NormGauss2D(name="gline")
gline.fwhm = 2.355
gline.fwhm.freeze()
gline.xpos = 0  # Freeze one dimension at correct position, this represents the cross-dispersion direction
gline.xpos.frozen = True

fit_model = bkg + gline

In [ ]:
x0 = np.random.normal(size=10000)
x1 = np.random.normal(size=10000)
hist, x0edges, x1edges = np.histogram2d(x0, x1, bins=(x0range, x1range))


In [ ]:
x0range = np.arange(-5, 5.001, 0.5)
x1range = np.arange(-5, 5.001, 0.5)
x0lo, x1lo = np.meshgrid(x0edges[:-1], x1edges[:-1])
x0hi, x1hi = np.meshgrid(x0edges[1:], x1edges[1:])

In [ ]:
def simulate_photons(n_line, n_bkg):
    lx0 = np.random.normal(size=n_line, loc=0)
    lx1 = np.random.normal(size=n_line, loc=0)
    bx0 = np.random.uniform(size=n_bkg, low=-5, high=5)
    bx1 = np.random.uniform(size=n_bkg, low=-5, high=5)
    hist, x0edges, x1edges = np.histogram2d(np.hstack([lx0, bx0]), 
                                            np.hstack([lx1, bx1]), 
                                            bins=(x0range, x1range))
    return hist

In [ ]:
image = DataIMGInt(
    "binned_image",
    x0lo=x0lo.flatten(),
    x1lo=x1lo.flatten(),
    x0hi=x0hi.flatten(),
    x1hi=x1hi.flatten(),
    y=hist.flatten(),
    shape=hist.shape,
)

In [ ]:
# Tunr te parameters until I'm close, because I'm guessing te coordinate system here
bkg.c0 = 0
#bkg.c0.freeze()

gline.ampl = 115
gline.ypos = 0.2 # start a bit off so I see that the fit actually did something


In [ ]:
f2d = Fit(data=image, model=fit_model, stat=Cash(), method=LevMar())

In [ ]:
fit_model

In [ ]:
spec1d = hist[8:12, :].sum(axis=0)
backg1d = hist[:5, :].sum(axis=0) + hist[15:, :].sum(axis=0)

dspec1d = Data1DInt(name="spec1d", xlo=x0range[:-1], xhi=x0range[1:], y=spec1d)
dbkg1d = Data1DInt(name="bkg1d", xlo=x0range[:-1], xhi=x0range[1:], y=backg1d)

bkg1d = Const1D(name="bkg1d")
gline1d = NormGauss1D(name="gline1d")
gline1d.fwhm = 2.355
gline1d.fwhm.freeze()


d1d = DataSimulFit('bothdata', (dspec1d, dbkg1d))
m1d = SimulFitModel('bothmodels', (bkg1d + gline1d, bkg1d))



In [ ]:
f1d = Fit(d1d, m1d, stat=Cash(), method=LevMar())

In [ ]:
hist  = simulate_photons(20, 40)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(figsize=(5, 4))
ax.imshow(
    hist,
    origin="lower",
    cmap="hot_r",
    # uneven number here to account for bin width, need start-end of bin
    # nor middle, also git steps-post, need to match
    # For this prototype, I just fiddled by hand until it looked right
    extent=[21.4, 21.8245, -55, 49],
    aspect="auto",
    # interpolation="gaussian",
)

ax.axhspan(-12, 8, color="blue", alpha=0.3)
ax.axhspan(-55, -27, color="0.3", alpha=0.3)
ax.axhspan(25, 50, color="0.3", alpha=0.3)

ax.set_xlabel("Wavelength (Ang)")
ax.set_ylabel("Cross-dispersion (arcsec)")

# create new axes on the right and on the top of the current axes
divider = make_axes_locatable(ax)
# below height and pad are in inches
ax_histy = divider.append_axes("right", 0.5, pad=0.1, sharey=ax)
# make some labels invisible
ax_histy.yaxis.set_tick_params(labelleft=False)

# Sum up to -6, because we set the xlim below
# that wat the count shown in the image and the counts in the histogram match
out = ax_histy.plot(
    hist[:, :-6].sum(axis=1), np.linspace(-50, 50, 20), drawstyle="steps-post"
)
ax_histy.set_xlabel("Counts")


ax_histx = divider.append_axes("top", 0.5, pad=0.1, sharex=ax)
# make some labels invisible
ax_histx.xaxis.set_tick_params(labelbottom=False)

out = ax_histx.plot(
    np.linspace(21.4, 21.8, 20), hist.sum(axis=0), drawstyle="steps-post"
)
ax_histx.set_ylabel("Counts")

ax.set_xlim(21.4, 21.69)
fig.savefig("../ADAP_proposal/dummy2dimage.pdf", dpi=300, bbox_inches="tight")

In [ ]:
fit2dres = np.zeros((1000, 9))
fit1dres = np.zeros((1000, 9))

for i in range(fit2dres.shape[0]):
    hist = simulate_photons(20, 40)
    image.set_dep(hist.flatten())

    # Reset to the starting values
    gline.ampl = 15
    gline.ypos = 0.2  # start a bit off so I see that the fit actually did something

    f2d.fit()
    fit_res = f2d.est_errors()
    for j, p in enumerate(fit_res.parnames):
        fit2dres[i, j * 3 : j * 3 + 3] = [
            fit_res.parvals[j],
            fit_res.parmins[j],
            fit_res.parmaxes[j],
        ]

    # Use the same simulated data for a 1d fit
    spec1d = hist[8:12, :].sum(axis=0)
    f1d.data.datasets[0].set_dep(spec1d)
    bkg1d = hist[:5, :].sum(axis=0) + hist[15:, :].sum(axis=0)
    f1d.data.datasets[1].set_dep(bkg1d)

    # reset starting values
    gline1d.ampl = 15
    gline1d.pos = 0.1
    f1d.fit()
    fit_res = f1d.est_errors()
    for j, p in enumerate(fit_res.parnames):
        fit1dres[i, j * 3 : j * 3 + 3] = [
            fit_res.parvals[j],
            fit_res.parmins[j],
            fit_res.parmaxes[j],
        ]

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)

axes[0].violinplot([fit2dres[:, 6], fit1dres[:, 6] / 0.68], showmeans=True)

# WE simulate a range of 10, whic hsi map to wave=21.4-21.8
axes[1].violinplot(
    [fit2dres[:, 3] / 10 * 0.4, fit1dres[:, 3] / 10 * 0.4], showmeans=True
)

for ax in axes:
    ax.yaxis.grid(True)
    ax.set_xticks([y + 1 for y in range(2)], labels=["2D fit", "1D fit"])
axes[0].set_ylabel("Number of photons")
axes[1].set_ylabel("position error (Ang)")

axes[0].axhspan(19.8, 20.2, color="k")
axes[1].axhspan(-0.0005, 0.0005, color="k")
axes[0].set_title('Flux')
axes[1].set_title('Position error')

fig.subplots_adjust(wspace=0.4)

fig.savefig("../ADAP_proposal/dummy2vilin.pdf", dpi=300, bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)

axes[0].violinplot([fit2dres[:, 6], fit1dres[:, 6] / 0.68], showmeans=True)

axes[1].violinplot(
    [fit2dres[:500, 6] / fit2dres[500:, 6], 
     fit1dres[:500, 6] / fit1dres[500:, 6]],
    showmeans=True,
)

for ax in axes:
    ax.yaxis.grid(True)
    ax.set_xticks([y + 1 for y in range(2)], labels=["2D fit", "1D fit"])
axes[0].set_ylabel("Number of photons")
axes[1].set_ylabel("Line ratio")

axes[0].axhspan(19.8, 20.2, color="k")
axes[1].axhspan(0.99, 1.01, color="k")
axes[0].set_title("Line flux")
axes[1].set_title("Ratio of two lines")

fig.subplots_adjust(wspace=0.4)

#fig.savefig("../ADAP_proposal/dummy2vilin.pdf", dpi=300, bbox_inches="tight")

In [ ]:
from scipy.stats import norm

In [ ]:
norm.ppf(0.75)

In [ ]:
FWHM = 2.355 sigma

In [ ]:
2.355 / (norm.ppf(0.75) * 2)